# Machine Learning Attack on Arbiter PUF
## Justin Schubeck, Alex Liu, Yash Bhat
### EEE5716 - Introduction to Hardware Security and Trust: Fall 2022
---

Import necessary packages.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Read in the data, and format to be [-1, 1] instead of [0, 1].

In [2]:
df = pd.read_excel('CRPSets.xls',header=None)
df = df.replace(0,-1)
df

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,-1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,-1,-1,-1,1,-1,1,1,1
1,1,-1,1,-1,1,-1,-1,-1,1,-1,...,1,1,1,-1,-1,-1,-1,-1,1,-1
2,1,1,-1,1,-1,1,1,-1,1,-1,...,1,-1,1,1,1,-1,-1,1,-1,-1
3,-1,1,-1,-1,-1,-1,1,1,-1,1,...,1,1,1,1,1,-1,-1,1,1,-1
4,1,1,-1,-1,-1,1,1,1,-1,1,...,-1,1,-1,1,1,1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,-1,1,-1,1,1,1,-1,-1,1,-1,...,1,-1,-1,-1,-1,1,1,-1,-1,-1
11996,-1,1,-1,-1,1,1,-1,1,-1,1,...,1,1,1,-1,-1,1,-1,1,-1,1
11997,1,1,1,-1,-1,-1,1,1,-1,-1,...,1,1,-1,1,-1,1,1,-1,-1,-1
11998,-1,1,-1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,1,-1,1,-1,-1,1


Separate the challenges from responses, and extract features.

In [3]:
challenges = df.to_numpy()[:,:64]
responses = df.to_numpy()[:,64]
features = np.cumprod(np.fliplr(challenges), axis=1, dtype=np.int8)

In [5]:
X_train, X_test, t_train, t_test = train_test_split(features, 
                                                    responses,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=7,
                                                    stratify=responses) 

In [6]:
%%time
param_grid = {'C':np.logspace(-3,3,7),
              'kernel':['linear','rbf']
              'gamma':list(np.logspace(-3,3,7))+['scale','auto']}

cv = StratifiedKFold(n_splits=5, 
                     shuffle=True,
                     random_state=7)
grid_search = GridSearchCV(SVC(), 
                           param_grid=param_grid,
                           cv=cv,
                           scoring='accuracy',
                           refit=True,
                           verbose=1,
                           n_jobs=-1)

grid_search.fit(X_train, t_train);
y_train = grid_search.best_estimator_.predict(X_train)
print('The best parameters of the SVM model are: ',grid_search.best_params_)
print('The SVM accuracy in training is: ',accuracy_score(t_train, y_train))
y_test = grid_search.best_estimator_.predict(X_test)
print('The SVM accuracy in test is: ',accuracy_score(t_test, y_test))

Fitting 3 folds for each of 63 candidates, totalling 189 fits
The SVM accuracy in training is:  0.999375
The best parameters of the SVM model are:  {'C': 100.0, 'gamma': 0.001}
